In [1]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
from matplotlib import pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image
from numpy import *

In [2]:
pos_path = r"images\train\pos"
neg_path= r"images\train\neg"

In [3]:
pos_list = os.listdir(pos_path)
neg_list = os.listdir(neg_path)
num_pos = size(pos_list)
num_neg = size(neg_list)
num_pos, num_neg

(4256, 4256)

In [4]:
orientations = 9
pixels_per_cell = (16, 16)
cells_per_block = (1, 1)
threshold = .3

#### get hog feather

In [5]:
data = []
labels = []
for file in pos_list:
    img = Image.open(pos_path + '\\' + file)
    img = img.resize((100,100))
    gray = img.convert('L')
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    data.append(fd)
    labels.append(1)
for file in neg_list:
    img= Image.open(neg_path + '\\' + file)
    img = img.resize((100,100))
    gray= img.convert('L')
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)
    data.append(fd)
    labels.append(0)

In [6]:
le = LabelEncoder()
labels = le.fit_transform(labels)

#### split train set

In [7]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(data), labels, test_size=0.2, random_state=42)

#### train using SVM

In [8]:
model = LinearSVC()
model.fit(trainData, trainLabels)

LinearSVC()

In [9]:
predictions = model.predict(testData)
print(classification_report(testLabels, predictions))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       864
           1       0.96      0.96      0.96       839

    accuracy                           0.96      1703
   macro avg       0.96      0.96      0.96      1703
weighted avg       0.96      0.96      0.96      1703



In [10]:
cnf = confusion_matrix(testLabels, predictions, labels=[0,1])
cnf

array([[832,  32],
       [ 33, 806]], dtype=int64)

In [11]:
acc = accuracy_score(testLabels, predictions)
precision = precision_score(testLabels, predictions, average='macro')
recall = recall_score(testLabels, predictions, average='macro')
print("Acc: ", acc)
print("Precision: ", precision)
print("Recall: ", recall)

Acc:  0.9618320610687023
Precision:  0.9618317767323796
Recall:  0.9618152121131859


#### F2 score

In [13]:
FN = cnf[0][0]
FP = cnf[0][1]
TN = cnf[1][0]
TP = cnf[1][1]
f2_score = ((1+4)*TP) / ((1+4)*TP + 4*FN + FP)
print("F2 Score: ", f2_score)

F2 Score:  0.5453315290933695


#### save model

In [14]:
joblib.dump(model, 'svm.npy')

['svm.npy']